## PRE-PROCESSING SOCIO-ECONOMIC PAAVO FILES 

Paavo-database is maintained by Statistics Finland and it contains socio-economic variables by postal code areas.
All files used in this analysis are downloaded from here: https://pxnet2.stat.fi/PXWeb/pxweb/fi/Postinumeroalueittainen_avoin_tieto

Variables chosen for analysis: 
- number of academic (lower + higher) degree holder in an area (divided by adult population to get percentage, 2018
- number of inhabitants belonging to highest and lowest income categories (divided by total population), 2017
- number of Inhabitants, 2018
- average and median incomes, 2017

The data was retrieved in late 2020 and these stats were the newest available then.

In [ ]:
import pandas as pd

#import separate csv files, downloaded from https://pxnet2.stat.fi/PXWeb/pxweb/fi/Postinumeroalueittainen_avoin_tieto
edu = pd.read_csv("/Users/kosokoso/Desktop/YLLI/edu2018.csv", encoding="latin8", skiprows=[0], na_values=["..."])
income = pd.read_csv("/Users/kosokoso/Desktop/YLLI/income2018.csv", encoding="latin8", skiprows=[0], na_values=["..."])
pop = pd.read_csv("/Users/kosokoso/Desktop/YLLI/pop2018.csv", encoding="latin8", skiprows=[0], na_values=["..."])
median_income = pd.read_csv("/Users/kosokoso/Desktop/YLLI/mediaanitulot.csv", encoding="latin8", skiprows=[0, 1], na_values=["..."])

In [50]:
#merge dataframes to one
df = edu.merge(income, on="Postal code area")
df1 = df.merge(pop, on="Postal code area")
data = df1.merge(median_income, on="Postal code area")
data

,Postal code area,"Aged 18 or over, total, 2018 (KO)","Academic degree - Lower level university degree, 2018 (KO)","Academic degree - Higher level university degree, 2018 (KO)","Inhabintants belonging to the lowest income category, 2017 (HR)","Inhabintants belonging to the highest income category, 2017 (HR)","Inhabitants, total, 2018 (HE)",Inhabitants' average income 2017 (HR),Inhabitants' median income 2017 (HR)
0,00100 Helsinki Keskusta - Etu-Töölö (Helsinki),16273,2983.0,5988.0,2772.0,6609.0,18427,42196.0,27577.0
1,00120 Punavuori (Helsinki),6202,1040.0,2205.0,1096.0,2566.0,7161,41657.0,27523.0
2,00130 Kaartinkaupunki (Helsinki),1319,190.0,519.0,246.0,618.0,1523,57766.0,30479.0
3,00140 Kaivopuisto - Ullanlinna (Helsinki),6800,1167.0,2480.0,1127.0,3078.0,7921,53555.0,29439.0
4,00150 Eira - Hernesaari (Helsinki),8304,1512.0,2793.0,1431.0,3224.0,9385,41564.0,26546.0
...,...,...,...,...,...,...,...,...,...
163,02860 Siikajärvi (Espoo),489,48.0,70.0,91.0,164.0,623,28707.0,25119.0
164,02920 Niipperi (Espoo),3825,595.0,689.0,548.0,1470.0,5577,34733.0,28034.0
165,02940 Lippajärvi-Järvenperä (Espoo),7704,1221.0,1557.0,1102.0,2838.0,10308,31043.0,27122.0
166,02970 Kalajärvi (Espoo),2755,399.0,423.0,414.0,1028.0,3824,30706.0,27202.0


In [51]:
#create new column that holds only postcode (key)
data["Posno"] = None

for i in range(len(data)):
    data.at[i, "Posno"] = data["Postal code area"].str.split(" ")[i][0]


In [52]:
#create new column and calculate the percentage of adult population holding academic degrees
data["hi_ed"] = (data["Academic degree - Lower level university degree, 2018 (KO)"].astype(float)+ data["Academic degree - Higher level university degree, 2018 (KO)"].astype(float)) / data["Aged 18 or over, total, 2018 (KO)"] * 100

#print post code areas with least and most degree holders and the mean to verify
print(data["hi_ed"].min(), data.loc[data["hi_ed"]== data["hi_ed"].min(), "Postal code area"])
print(data["hi_ed"].max(), data.loc[data["hi_ed"]== data["hi_ed"].max(), "Postal code area"])
print(data["hi_ed"].mean())

10.344827586206897 115    01720  Petikko (Vantaa)
Name: Postal code area, dtype: object
57.91457286432161 127    02160  Westend (Espoo)
Name: Postal code area, dtype: object
36.674546709533004


In [53]:
#crete new columns for percentage of people belonging to the highest and lowest income categories
data["low_in"] = data["Inhabintants belonging to the lowest income category, 2017 (HR)"] / data["Inhabitants, total, 2018 (HE)"] * 100
data["high_in"] = data["Inhabintants belonging to the highest income category, 2017 (HR)"] / data["Inhabitants, total, 2018 (HE)"] * 100

#drop unnecessary columns
data = data.drop(["Aged 18 or over, total, 2018 (KO)", "Academic degree - Lower level university degree, 2018 (KO)",
                 "Academic degree - Higher level university degree, 2018 (KO)", "Inhabintants belonging to the lowest income category, 2017 (HR)",
                  "Inhabintants belonging to the highest income category, 2017 (HR)"],axis=1)


data.rename(columns = {"Inhabitants, total, 2018 (HE)":"pop", 
                              "Inhabitants' average income 2017 (HR)":"avg_income",
                              "Inhabitants' median income 2017 (HR)":"median_income"}, inplace = True)
data

,Postal code area,pop,avg_income,median_income,Posno,hi_ed,low_in,high_in
0,00100 Helsinki Keskusta - Etu-Töölö (Helsinki),18427,42196.0,27577.0,00100,55.128126,15.043143,35.865849
1,00120 Punavuori (Helsinki),7161,41657.0,27523.0,00120,52.321832,15.305125,35.832984
2,00130 Kaartinkaupunki (Helsinki),1523,57766.0,30479.0,00130,53.752843,16.152331,40.577807
3,00140 Kaivopuisto - Ullanlinna (Helsinki),7921,53555.0,29439.0,00140,53.632353,14.228002,38.858730
4,00150 Eira - Hernesaari (Helsinki),9385,41564.0,26546.0,00150,51.842486,15.247736,34.352690
...,...,...,...,...,...,...,...,...
163,02860 Siikajärvi (Espoo),623,28707.0,25119.0,02860,24.130879,14.606742,26.324238
164,02920 Niipperi (Espoo),5577,34733.0,28034.0,02920,33.568627,9.826071,26.358257
165,02940 Lippajärvi-Järvenperä (Espoo),10308,31043.0,27122.0,02940,36.059190,10.690726,27.532014
166,02970 Kalajärvi (Espoo),3824,30706.0,27202.0,02970,29.836661,10.826360,26.882845


In [54]:
#save to csv
data.to_csv("/Users/kosokoso/Desktop/YLLI/sos_econ.csv")
#joined to shapefile of post code areas in QGIS